# Preprocess currents for simulations

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import os
import sys

In [ ]:
pythoncodepath = os.path.abspath(os.path.join('..', '_pythoncode'))
sys.path = [pythoncodepath] + sys.path
import importhelper
importhelper.addfolders2path(pythoncodepath)

In [ ]:
import data_utils

# Read Currents and create current density files.

In [ ]:
data_utils.make_dir("PreprocessedData")

## Global parameters

In [ ]:
# Parameters to correct units.
A_elec = np.pi * (15e-6)**2
ee_elec = 70e-6

Areas = {}

Areas['1x1'] = A_elec
for A in [2,4,8,10,16,20]:
    AxA = str(A) + 'x' + str(A)
    Areas[AxA] = (ee_elec * A)**2
    
for A in [1,2,4,8,10,16,20]:
    AxA = str(A) + 'x' + str(A)
    print(AxA + ":\t {:.0f} um²".format(1e12*Areas[AxA]))

## Read and preprocess data.

In [ ]:
# Input file.
I = {} # Current
J = {} # Current density
Q = {} # Charge
X = {} # Charge density

for A in [1,2,4,8,10,16,20]:
    AxA = str(A) + 'x' + str(A)

    # Read currents.
    input_file = os.path.join('ExperimentalData', AxA + '-biphasische stimualtionsstrom.dat')
    
    # Create table.
    current_table         = pd.read_csv(input_file, sep='\t')
    current_table['ms']   = current_table*1e-3
    current_table.columns = ['Time [s]'] + ['I' + str(i) + ' [A]' for i in range(current_table.columns.size-1)]     
    I[AxA] = current_table
    
    dt = np.mean(current_table['Time [s]'].values[1:] - current_table['Time [s]'].values[:-1])
    
    # Compute charge.
    Q[AxA] = {}
    for i in range(len(current_table.columns[1:])):
        idx_on = 0#np.where(current_table['Time [s]'] >= 3e-3)[0][0]
        idx_off = -1#np.where(current_table['Time [s]'] >= 5e-3)[0][0]
        
        Q[AxA]['I'+str(i)] = np.mean(np.clip(current_table['I'+str(i)+' [A]'], 0, None)) * current_table['Time [s]'].max()
    
    # Compute current density.
    current_density_table = current_table.copy()
    current_density_table.iloc[:,1:] /= Areas[AxA]
    current_density_table.columns = ['Time [s]'] + ['J' + str(i) + ' [A/m^2]' for i in range(current_density_table.columns.size-1)]
    J[AxA] = current_density_table
        
    
    # Compute charge density.
    X[AxA] = {}
    for i in range(len(current_table.columns[1:])):
        X[AxA]['I'+str(i)] = Q[AxA]['I'+str(i)] / Areas[AxA]
        
data_utils.save_var(I, os.path.join('PreprocessedData', 'from_raw_I.pkl'))
#data_utils.save_var(J, os.path.join('PreprocessedData', 'from_raw_J.pkl'))
#data_utils.save_var(Q, os.path.join('PreprocessedData', 'from_raw_Q.pkl'))
data_utils.save_var(X, os.path.join('PreprocessedData', 'from_raw_X.pkl'))

# Plot

In [ ]:
# Plot
plt.figure(1, figsize = (14, 14))

idx=1
for AxA in ['1x1','2x2','4x4','8x8','10x10','16x16','20x20']:
    plt.subplot(3,3,idx)
    plt.title(AxA)
    plt.xlabel('Time [s]')
    plt.ylabel('I [uA]')
    c = 1
    for i in I[AxA].columns[1:]:
        plt.plot(I[AxA]['Time [s]'], I[AxA][i]*1e6)
        c+=1
    plt.legend(I[AxA].columns[1:])
    idx+=1
plt.tight_layout()
plt.show()

# Save current densities

Current files will be used in COMSOL to simulate the currents

In [ ]:
A_list = ['1x1', '2x2','4x4','10x10']

In [ ]:
# Save currents to files.
for AxA in A_list: 
    current = I[AxA].iloc[:,1:].values.T.flatten()
    time = np.tile(I[AxA]['Time [s]'], I[AxA].shape[1]-1)
    gain = np.repeat(np.arange(I[AxA].shape[1]-1), I[AxA].shape[0])
    
    comsol_stimfile = pd.DataFrame(np.stack([time, gain, current]).T)
                     
    filename = str(AxA) + '/I_stim.txt'
    comsol_stimfile.to_csv(filename, index=False, header=None)